# Imports and configs

In [1]:
!pip install -q autogluon.tabular ray==2.10.0

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.15.1 requires botocore<1.35.24,>=1.35.16, but you have botocore 1.29.165 which is incompatible.
bigframes 0.22.0 requires google-cloud-bigquery[bqstorage,pandas]>=3.10.0, but you have google-cloud-bigquery 2.34.4 which is incompatible.
bigframes 0.22.0 requires google-cloud-storage>=2.0.0, but you have google-cloud-storage 1.44.0 which is incompatible.
bigframes 0.22.0 requires pandas<2.1.4,>=1.5.0, but you have pandas 2.2.3 which is incompatible.
cesium 0.12.3 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
dataproc-jupyter-plugin 0.1.79 requires pydantic~=1.10.0, but you have pydantic 2.9.2 which is incompatible.
libpysal 4.9.2 requires packaging>=22, but you have packaging 21.3 which is incompatible.
libpysal 4.9.2 requires shapely>=2.0.1, but you have shapely 1.8.5.post

In [2]:
from autogluon.tabular import TabularPredictor
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import warnings
import shutil

warnings.filterwarnings('ignore')

In [3]:
class CFG:
    train_path = '/kaggle/input/playground-series-s4e11/train.csv'
    test_path = '/kaggle/input/playground-series-s4e11/test.csv'
    sample_sub_path = '/kaggle/input/playground-series-s4e11/sample_submission.csv'
    original_data_path = '/kaggle/input/depression-surveydataset-for-analysis/final_depression_dataset_1.csv'
    
    target = 'Depression'
    time_limit = 3600 * 5

# Loading data

In [4]:
train = pd.read_csv(CFG.train_path, index_col='id')
test = pd.read_csv(CFG.test_path, index_col='id')

original = pd.read_csv(CFG.original_data_path)
original[CFG.target] = original[CFG.target].map({'Yes': 1, 'No': 0})

# Fitting the predictor

In [5]:
predictor = TabularPredictor(
    problem_type='binary',
    eval_metric='accuracy',
    label=CFG.target,
    verbosity=2
)

No path specified. Models will be saved in: "AutogluonModels/ag-20241102_083542"


In [6]:
predictor.fit_pseudolabel(
    train_data=train,
    pseudo_data=original,
    time_limit=CFG.time_limit,
    keep_only_best=True,
    presets='best_quality',
    ag_args_fit={
#         'num_gpus': 1, 
        'num_cpus': 4
    }
)

Predictor not fit prior to pseudolabeling. Fitting now...
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.14
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu Jun 27 20:43:36 UTC 2024
CPU Count:          4
Memory Avail:       30.24 GB / 31.36 GB (96.4%)
Disk Space Avail:   19.50 GB / 19.52 GB (99.9%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdou

In [7]:
predictor.leaderboard(silent=True).style.background_gradient(subset=['score_val'], cmap='RdYlGn')

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.940924,accuracy,20.438312,7924.355096,0.014924,7.383153,2,True,10
1,CatBoost_r177_BAG_L1,0.940028,accuracy,0.363356,503.840276,0.363356,503.840276,1,True,7
2,NeuralNetFastAI_r191_BAG_L1,0.940014,accuracy,5.347926,2516.121604,5.347926,2516.121604,1,True,9
3,XGBoost_BAG_L1,0.939801,accuracy,1.025445,398.187296,1.025445,398.187296,1,True,5
4,CatBoost_BAG_L1,0.939680,accuracy,0.370881,581.392864,0.370881,581.392864,1,True,3
5,LightGBMXT_BAG_L1,0.939645,accuracy,0.808687,71.244358,0.808687,71.244358,1,True,1
6,NeuralNetTorch_r79_BAG_L1,0.939339,accuracy,1.256872,1199.607165,1.256872,1199.607165,1,True,8
7,NeuralNetFastAI_BAG_L1,0.939048,accuracy,3.815850,1762.468336,3.815850,1762.468336,1,True,4
8,NeuralNetTorch_BAG_L1,0.938571,accuracy,1.095546,848.375237,1.095546,848.375237,1,True,6
9,RandomForestGini_BAG_L1,0.937178,accuracy,6.338826,35.734806,6.338826,35.734806,1,True,2


# Creating a submission file

In [8]:
sub = pd.read_csv(CFG.sample_sub_path)
sub[CFG.target] = predictor.predict(test).values
sub.to_csv('sub_autogluon.csv', index=False)
sub

,id,Depression
0,140700,0
1,140701,0
2,140702,0
3,140703,1
4,140704,0
...,...,...
93795,234495,0
93796,234496,1
93797,234497,0
93798,234498,1


In [9]:
shutil.rmtree("AutogluonModels")